# FT-MS质谱数据的初步处理

本笔记本将原论坛脚本（单行、面向 ipynb）整理为结构化的 Jupyter Notebook。
- 保留并补充了原有中文注释；
- 数据文件位于 `data/` 目录，使用 Excel (`ICRMS-1.xlsx` 至 `ICRMS-4.xlsx`)；
- 使用 `pykrev` 完成元素计数、比值、DBE、芳香性指数、Kendrick 质量缺陷、Van Krevelen 图等分析。


## 1. 导入依赖
确保已经在项目根目录创建了虚拟环境 `.venv` 并安装了依赖（已在 README 提供指令）。


In [ ]:
# 首先导入我们需要的库
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import sys, types
# # 构造一个假模块 numpy.lib.function_base，只提供 pykrev 需要的对象
# mod = types.ModuleType('numpy.lib.function_base')
# mod.rot90 = np.rot90
# mod._rot90_dispatcher = lambda m, k=1, axes=(0,1): (m,)  # 占位即可
# sys.modules['numpy.lib.function_base'] = mod
# import pykrev as pk

# 安全补丁：只在真实的 numpy.lib.function_base 上补缺失的符号
import numpy.lib.function_base as nlfb
    # 某些旧代码引用了私有的 _rot90_dispatcher；新版本可能没有
if not hasattr(nlfb, "_rot90_dispatcher"):
    def _rot90_dispatcher(m, k=1, axes=(0,1)):
        return (m,)
    nlfb._rot90_dispatcher = _rot90_dispatcher
# 理论上 rot90 仍在 numpy 顶层与 numpy.lib.function_base；但稳妥起见：
if not hasattr(nlfb, "rot90"):
    nlfb.rot90 = np.rot90
import pykrev as pk


from pathlib import Path

# 一些显示设置（可选）
pd.set_option('display.max_rows', 10)
pd.set_option('display.max_columns', None)
plt.rcParams['figure.dpi'] = 120


## 2. 读取数据（来自 data/ 目录的 Excel 文件）
原脚本使用 `pk.read_csv('F:/example_A.csv')`，本项目数据存放在 `data/` 目录下的 Excel 文件中。
文件列名：`Formula`, `m/z`, `Intensity`。为与 `pykrev` 习惯统一，这里转换为小写：`formula`, `mz`, `intensity`。


In [ ]:
from pathlib import Path
import os
print("cwd =", Path.cwd())
print("cwd(list) =", [p.name for p in Path.cwd().iterdir()][:20])
print("data exists?", Path("data").exists(), "is_dir?", Path("data").is_dir())
if Path("data").exists():
    print("data list =", [repr(p.name) for p in Path("data").iterdir()])


In [ ]:
os.chdir("/home/rwi2013")

In [ ]:
# 1) 先从 Excel 转成 CSV（确保列名正好是 formula, intensity, mz）
xlsx = Path("data/ICRMS-1.xlsx")
df = pd.read_excel(xlsx)
df = df.rename(columns={
    'Formula':'formula', 'FORMULA':'formula',
    'Intensity':'intensity', 'INTENSITY':'intensity',
    'mz':'mz', 'm/z':'mz', 'MZ':'mz'
})
df[['formula','intensity','mz']].to_csv(xlsx.with_suffix(".csv"), index=False)

# 2) 用 pykrev 读 CSV（会得到 msTuple）
A = pk.read_csv(xlsx.with_suffix(".csv"), column_headers=True)
print(type(A))       # -> <class 'pykrev.formula.msTuple.msTuple'>
A.summary()          # 现在能用了
print(type(A.formula))
print(type(A.intensity))

In [ ]:
#如果需要根据mz或者intensity进行过滤，运行如下代码
#A = A.filter_mz(100,1000)
#A = A.filter_intensity(1e6,1e7)

In [ ]:
#计算每个化学式中C、H、N、O、P、S、Cl和F的数量
elementCounts = pk.element_counts(A)
print(elementCounts)

In [ ]:
# HC、OH、OC 比
elementRatios = pk.element_ratios(A, ratios=['HC','OH','OC'])

# 双键等效值 DBE
dbe = pk.double_bond_equivalent(A)

# 芳香性指数（rAI）
ai = pk.aromaticity_index(A, index_type='rAI')

# 碳的标称氧化态（NOSC）
nosc = pk.nominal_oxidation_state(A)

In [ ]:
# 标称质量 / 平均质量 / 精确单同位素质量
nominalMass = pk.calculate_mass(A, method='nominal')
averageMass = pk.calculate_mass(A, method='average')
monoisotopicMass = pk.calculate_mass(A, method='monoisotopic')

# 去质子化分析物的预期质量（示例：负一价、质子化修正）
massExpected = pk.calculate_mass(A, method='monoisotopic', ion_charge=-1, protonated=True)

# 相对质量误差（ppm），单位为ppm
# A 如果是 msTuple，通常有 A.mz；若没有，就改为你实际的 m/z 序列
massError = (massExpected - A.mz) / A.mz * 1e6
display(pd.Series(massError).head())


In [ ]:
#计算一组校准的m/z值的Kendrick质量和Kendrick质量缺陷。
kendrickMass, kendrickMassDefect = pk.kendrick_mass_defect(A, base=['CH2'], rounding='even')

In [ ]:
fig, ax = pk.van_krevelen_plot(A, y_ratio='HC', c=dbe, s=7, cmap='plasma')
# 取出散点 artist 以挂接 colorbar（collections[0] 通常是散点）
scatter = ax.collections[0]
cbar = plt.colorbar(scatter, ax=ax)
cbar.set_label('Double Bond Equivalence (DBE)')
plt.show()


In [ ]:
import matplotlib.pyplot as plt

# 5A. 密度着色（Kernel Density）
plt.figure()
pk.van_krevelen_plot(A, c='density', s=7)
plt.colorbar().set_label('Kernel Density')
plt.grid(False)
plt.show()

# 5B. 直方图热图（等宽分箱）
plt.figure()
fig, ax, d_index = pk.van_krevelen_histogram(A, bins=[10, 10], cmap='viridis')
plt.colorbar().set_label('Counts')
plt.show()

# 5C. 直方图热图（自定义边界）
import numpy as np
plt.figure()
fig, ax, d_index = pk.van_krevelen_histogram(
    A,
    bins=[np.linspace(0, 1, 5), np.linspace(0, 2, 5)],
    cmap='cividis'
)
plt.colorbar().set_label('Counts')
plt.show()


In [ ]:
plt.figure(figsize=(10, 7))
fig, ax, (kendrickMass, kendrickMassDefect) = pk.kendrick_mass_defect_plot(
    A, base='CHON', rounding='ceil', s=3, c=ai
)
plt.colorbar().set_label('Aromaticity Index')
plt.xlim([0, 1000])
plt.show()

In [ ]:
plt.figure()
fig, ax, (mean, median, sigma) = pk.atomic_class_plot(
    A, element='O', color='b', summary_statistics=True, bins=range(0, 33)
)
plt.show()

In [ ]:
plt.figure()
fig, ax, (compounds, counts) = pk.compound_class_plot(A, color='g', method='MSCC')
plt.show()


In [ ]:
import matplotlib.pyplot as plt

# 9A. 单同位素质量直方图（含 KDE）
plt.figure()
fig, ax, (mean, median, sigma) = pk.mass_histogram(
    A,
    method='monoisotopic',
    bin_width=20,
    summary_statistics=True,
    color='blue',
    alpha=0.5,
    kde=True,
    kde_color='blue',
    density=False
)
plt.xlabel('Monoisotopic atomic mass (Da)')
plt.show()

# 9B. 另一种配置（仅 KDE）
plt.figure()
fig, ax, (mean, median, sigma) = pk.mass_histogram(
    A,
    method='me',
    kde=True,
    hist=False,
    kde_color='red',
    summary_statistics=True,
    deprotonated=True
)
plt.show()


In [ ]:
fig, ax1, ax2 = pk.mass_spectrum(
    A,
    method='mz',          # 关键改这里，用 measured m/z
    logTransform=False,
    stepSize=4,           # 这个小数位随便，mz 本身是唯一的
    lineColor='g',
    lineWidth=1.2,
    invertedAxis=massError   # 如果 massError 是基于 A.mz 算的，就能对齐
)
plt.show()

In [ ]:
plt.figure()
pk.spiral_plot(A)
plt.show()

plt.figure()
pk.spiral_plot(A, colour=dbe, mass_order='descending', theta=9999)
plt.show()


In [ ]:
A_diversity = pk.diversity_indices(A, verbose=True, indices=['SW','DBE','GS'])

# 香农-维纳（丰度多样性）
A_diversity['D_a_SW']

# 功能多样性（以 DBE 度量）
A_diversity['D_f_DBE']

# 基尼-辛普森（均匀度）
A_diversity['D_a_GS']
